In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [4]:
data = pd.read_csv('5500_Dataset2.csv')
data['Date'] = data['Year'].astype(str) + '-' + data['Month'].astype(str)
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m')
data.set_index('Date', inplace=True)
data = data.resample('M').last()
data.drop(['Year', 'Month'], axis=1, inplace=True)
etf_columns = data.columns[:11]
factors = data.columns[11:]
data[etf_columns] = data[etf_columns].pct_change()
data[factors] = data[factors].shift(2) # Create the lag for macro factors
data['CPI_lag1'] = data['CPI'].shift(3)
data['CPI_lag2'] = data['CPI'].shift(4)
data['CPI_lag3'] = data['CPI'].shift(5)
data = data.iloc[7:]
data.fillna(-100, inplace=True)

/var/folders/55/ysck04bj54j_6bw0v46q2k600000gn/T/ipykernel_41135/1407998261.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['CPI_lag1'] = data['CPI'].shift(3)
/var/folders/55/ysck04bj54j_6bw0v46q2k600000gn/T/ipykernel_41135/1407998261.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['CPI_lag2'] = data['CPI'].shift(4)
/var/folders/55/ysck04bj54j_6bw0v46q2k600000gn/T/ipykernel_41135/1407998261.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [5]:
def clean_feature_names(columns):
    return [col.replace(" ", "_")
                .replace(",", "")
                .replace("&", "and")
                .replace("-", "_")
                .replace("/", "_")
                .replace("__", "_")
            for col in columns]

In [6]:
data.columns = clean_feature_names(data.columns)

In [7]:
data

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,Unemp_Rate,Auto_Sales,Durable_Goods,NYMEX_Oil_Price,Michigan_CSI,GLD,Macro_2,CPI_lag1,CPI_lag2,CPI_lag3
Date,,,,,,,,,,,,,,,,,,,,,
2005-06-30,-0.019912,-100.000000,0.069554,0.012508,-0.026557,-0.013453,-0.019244,-100.000000,0.059829,-0.005336,...,5.4,1615.98,180157.0,51.75,94.1,43.35,-4.845874,191.70,191.70,190.80
2005-07-31,0.055305,-100.000000,0.071262,0.015442,0.038194,0.057143,0.031977,-100.000000,0.022333,0.022798,...,5.2,1542.05,186553.0,55.40,92.6,41.65,-4.863422,191.60,191.70,191.70
2005-08-31,-0.041711,-100.000000,0.063613,-0.016423,-0.023411,-0.009828,-0.017606,-100.000000,0.006675,-0.003497,...,5.2,1537.32,197735.0,49.72,87.7,43.44,-4.843102,192.40,191.60,191.70
2005-09-30,0.007254,-100.000000,0.062884,0.008658,0.016634,0.002481,0.014337,-100.000000,0.042194,-0.005702,...,5.1,1720.01,199935.0,51.97,86.9,42.82,-4.853319,193.10,192.40,191.60
2005-10-31,0.002216,-100.000000,-0.090354,0.030656,-0.014437,-0.022896,-0.004947,-100.000000,-0.064199,-0.030437,...,5.0,1845.71,186870.0,56.50,96.0,43.40,-4.850744,193.70,193.10,192.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,-0.045850,-0.046552,-0.009366,-0.041627,-0.035246,-0.057582,-0.011275,-0.084406,0.016540,-0.050133,...,3.7,1105.85,260363.0,71.33,69.7,189.31,-5.386920,307.53,307.29,306.19
2024-05-31,0.032680,0.069362,-0.003478,0.031702,0.016403,0.070764,0.024282,0.051433,0.089568,0.024056,...,3.7,1266.64,264841.0,75.85,79.0,205.72,-5.388810,308.02,307.53,307.29
2024-06-30,-0.030544,0.031526,-0.014177,-0.008952,-0.009862,0.078312,-0.002227,0.019513,-0.055408,0.018232,...,3.9,1476.40,267876.0,78.30,76.9,211.87,-5.440779,308.74,308.02,307.53


## XGB Base Feature

In [8]:
macro_factor_1 = ['GLD', 'FTSE', 'N225', 'Corprote Bond (BAA) Yield', 'US_1yr',
       'US_10yr', 'CPI', 'Housing Starts', 'Monetary Base', 'M2', 'Unemp_Rate',
       'Auto Sales', 'Durable Goods', 'NYMEX_Oil_Price', 'Michigan_CSI','CPI_lag1', 'CPI_lag2', 'CPI_lag3']
macro_factor_1= clean_feature_names(macro_factor_1)

In [9]:
window_size = 36  # 3 year rolling window
all_predictions = []
all_real_top4 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor_1]
    X_test = test_data[macro_factor_1].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
        model_1.fit(X_train, Y_train)
        predicted_returns[etf] = model_1.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    all_predictions.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})

    # actual rank top 4
    real_top4 = sorted(actual_returns, key=actual_returns.get, reverse=True)[:4]
    all_real_top4.append({'Date': test_data.name, 'Top1': real_top4[0], 'Top2': real_top4[1], 'Top3': real_top4[2], 'Top4': real_top4[3]})

# to DataFrame
xgb_df_1 = pd.DataFrame(all_predictions)
real_top4_df = pd.DataFrame(all_real_top4)

In [10]:
# Top-4 accuracy
top_4_accuracy = []
for i in range(len(xgb_df_1)):
    predicted_set = set(xgb_df_1.iloc[i, 1:5])  # # predicted top 4
    real_set = set(real_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # Top-4 accuracy
    top_4_accuracy.append(accuracy)

# average Top-4 accuracy
average_top_4_accuracy = sum(top_4_accuracy) / len(top_4_accuracy)
print("Average Top-4 Accuracy:", average_top_4_accuracy)

# print
xgb_df_1['Top-4 Accuracy'] = top_4_accuracy
print("Predictions with Top-4 Accuracy:")
print(xgb_df_1)

Average Top-4 Accuracy: 0.3801546391752577
Predictions with Top-4 Accuracy:
          Date Top1 Top2 Top3 Top4  Top-4 Accuracy
0   2008-07-31  XLK  XLU  XLI  XLY            0.25
1   2008-08-31  XLE  XLP  XLU  XLV            0.50
2   2008-09-30  XLK  XLV  XLU  XLF            0.50
3   2008-10-31  XLK  XLV  XLE  XLP            0.75
4   2008-11-30  XLY  XLP  XLF  XLU            0.50
..         ...  ...  ...  ...  ...             ...
189 2024-04-30  XLY  XLK  XLF  XLE            0.25
190 2024-05-31  XLY  XLB  XLI  XLF            0.00
191 2024-06-30  XLB  XLY  XLI  XLK            0.50
192 2024-07-31  XLK  XLC  XLI  XLF            0.50
193 2024-08-31  XLK  XLY  XLF  XLC            0.00

[194 rows x 6 columns]


In [11]:
# Features importance
importance_1 = model_1.get_booster().get_score(importance_type='gain')

importance_df1 = pd.DataFrame({
    'Feature': list(importance_1.keys()),
    'Importance': list(importance_1.values())
}).sort_values(by='Importance', ascending=False)

importance_df1

,Feature,Importance
10,Auto_Sales,0.003380
9,Unemp_Rate,0.002057
1,FTSE,0.001619
12,NYMEX_Oil_Price,0.001441
4,US_1yr,0.000897
2,N225,0.000818
0,GLD,0.000607
6,Housing_Starts,0.000512
8,M2,0.000492
13,Michigan_CSI,0.000299


In [40]:
xgb_df_1.to_csv('xbg_base.csv')

## XGB Engineered Features

In [13]:
macro_factor_2 = [
        'Manufacturers Computer And Eletronic Products Inventories Sales',
        'Monthly Imports Commodities',
        'Import Air Freight',
        'CPI for Urban Consumers All items',
        'CPI for Urban Consumers Energy',
        'CPI for Urban Consumers Housing',
        'PPI Manufacturing Industries',
        'PPI All Commodities',
        'Monthly Export price index Foods, feeds, and beverages',
        'Monthly Export price index Consumer goods, excluding automotives',
        'PPI Iron and steel mills and ferroalloy',
        'PPI Aircraft engine and engine parts',
        'PPI sub-sector data for Truck transportation',
        'PPI General medical and surgical hospitals',
        'Housing Starts']
macro_factor_2= clean_feature_names(macro_factor_2)


In [14]:
window_size = 36  # 3 year rolling window
all_predictions_2 = []
all_real_top4 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor_2]
    X_test = test_data[macro_factor_2].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, enable_categorical=True)
        model_2.fit(X_train, Y_train)
        predicted_returns[etf] = model_2.predict(X_test)[0]  # predict return

    # predict and rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    all_predictions_2.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})


# to DataFrame
xgb_df_2 = pd.DataFrame(all_predictions_2)

In [15]:
# Top-4 Accuracy
top_4_accuracy2 = []
for i in range(len(xgb_df_2)):
    predicted_set = set(xgb_df_2.iloc[i, 1:5])  # predicted top 4
    real_set = set(real_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # calculate accuracy
    top_4_accuracy2.append(accuracy)

# average accuracy
average_top_4_accuracy2 = sum(top_4_accuracy2) / len(top_4_accuracy2)
print("Average Top-4 Accuracy2:", average_top_4_accuracy2)

# print
xgb_df_2['Top-4 Accuracy'] = top_4_accuracy2
print("Predictions with Top-4 Accuracy2:")
print(xgb_df_2)

Average Top-4 Accuracy2: 0.4007731958762887
Predictions with Top-4 Accuracy2:
          Date Top1 Top2 Top3  Top4  Top-4 Accuracy
0   2008-07-31  XLK  XLF  XLU   XLB            0.25
1   2008-08-31  XLE  XLV  XLU   XLB            0.25
2   2008-09-30  XLV  XLU  XLP   XLB            0.50
3   2008-10-31  XLV  XLP  XLK   XLB            0.75
4   2008-11-30  XLY  XLE  XLV   XLI            0.50
..         ...  ...  ...  ...   ...             ...
189 2024-04-30  XLY  XLF  XLC   XLE            0.25
190 2024-05-31  XLE  XLF  XLI   XLU            0.25
191 2024-06-30  XLI  XLB  XLE   XLY            0.25
192 2024-07-31  XLI  XLC  XLK  XLRE            0.50
193 2024-08-31  XLK  XLC  XLV  XLRE            0.50

[194 rows x 6 columns]


In [16]:
# Features importance
importance_2 = model_2.get_booster().get_score(importance_type='gain')

importance_df2 = pd.DataFrame({
    'Feature': list(importance_2.keys()),
    'Importance': list(importance_2.values())
}).sort_values(by='Importance', ascending=False)

importance_df2

,Feature,Importance
10,PPI_Aircraft_engine_and_engine_parts,0.004604
1,Monthly_Imports_Commodities,0.001899
13,Housing_Starts,0.001580
7,Monthly_Export_price_index_Foods_feeds_and_bev...,0.001452
11,PPI_sub_sector_data_for_Truck_transportation,0.000976
6,PPI_All_Commodities,0.000959
9,PPI_Iron_and_steel_mills_and_ferroalloy,0.000848
0,Manufacturers_Computer_And_Eletronic_Products_...,0.000756
12,PPI_General_medical_and_surgical_hospitals,0.000751
2,Import_Air_Freight,0.000573


## XGB Base & Engineered Feature

In [17]:
macro_factor_3 = ['GLD', 'FTSE', 'N225', 'Corprote Bond (BAA) Yield', 'US_ 1yr',
        'US_10yr', 'CPI', 'Housing Starts', 'Monetary Base', 'M2', 'Unemp_Rate',
        'Auto Sales', 'Durable Goods', 'NYMEX_Oil_Price', 'Michigan_CSI',
        'Manufacturers Computer And Eletronic Products Inventories Sales',
        'Monthly Imports Commodities',
        'Import Air Freight',
        'CPI for Urban Consumers All items',
        'CPI for Urban Consumers Energy',
        'CPI for Urban Consumers Housing',
        'PPI Manufacturing Industries',
        'PPI All Commodities',
        'Monthly Export price index Foods, feeds, & beverages',
        'Monthly Export price index Consumer goods, excluding automotives',
        'PPI Iron and steel mills and ferroalloy',
        'PPI Aircraft engine and engine parts',
        'PPI sub-sector data for Truck transportation',
        'PPI General medical and surgical hospitals']

macro_factor_3= clean_feature_names(macro_factor_3)

In [18]:
window_size = 36  # 3 years rolling
all_predictions_3 = []

# rolling prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor_3]
    X_test = test_data[macro_factor_3].values.reshape(1, -1)  # testing features

    # predict etf returns
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual retuns

    params = {
    'max_depth': 4,  # Not too deep to prevent overfitting
    'learning_rate': 0.02,  # Small learning rate for better generalization
    'n_estimators': 400,  # Sufficient number of trees for small datasets
    'min_child_weight': 5,  # Avoid splitting too often
    'gamma': 1,  # Prevent overfitting
    'subsample': 0.8,  # Use 80% of data to introduce randomness
    'colsample_bytree': 0.8,  # Use 80% of features for each tree
    'lambda': 2,  # L2 regularization to control overfitting
    'alpha': 0,  # L1 regularization (use higher if you want to enforce sparsity)
    'objective': 'reg:squarederror',  # Assuming a regression problem
    'eval_metric': 'rmse'
}
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set target
        model_3 = xgb.XGBRegressor(**params)
        model_3.fit(X_train, Y_train)
        predicted_returns[etf] = model_3.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    all_predictions_3.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})


# to DataFrame
xgb_df_3 = pd.DataFrame(all_predictions_3)

In [19]:
# Top-4 Accuracy
top_4_accuracy3 = []
for i in range(len(xgb_df_3)):
    predicted_set = set(xgb_df_3.iloc[i, 1:5])  # Predicted top 4
    real_set = set(real_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set) 
    accuracy = match_count / 4  # Top-4 Accuracy
    top_4_accuracy3.append(accuracy)

# Average Top-4 Accuracy
average_top_4_accuracy3 = sum(top_4_accuracy3) / len(top_4_accuracy3)
print("Average Top-4 Accuracy:", average_top_4_accuracy3)


xgb_df_3['Top-4 Accuracy'] = top_4_accuracy3
print("Predictions with Top-4 Accuracy3:")
print(xgb_df_3)

Average Top-4 Accuracy: 0.38788659793814434
Predictions with Top-4 Accuracy3:
          Date Top1 Top2 Top3 Top4  Top-4 Accuracy
0   2008-07-31  XLE  XLB  XLU  XLI            0.25
1   2008-08-31  XLE  XLB  XLU  XLP            0.25
2   2008-09-30  XLE  XLB  XLU  XLP            0.25
3   2008-10-31  XLE  XLB  XLP  XLU            0.50
4   2008-11-30  XLB  XLE  XLP  XLI            0.50
..         ...  ...  ...  ...  ...             ...
189 2024-04-30  XLE  XLK  XLI  XLF            0.50
190 2024-05-31  XLE  XLK  XLI  XLF            0.25
191 2024-06-30  XLE  XLK  XLI  XLF            0.25
192 2024-07-31  XLE  XLK  XLI  XLU            0.50
193 2024-08-31  XLE  XLK  XLI  XLV            0.25

[194 rows x 6 columns]


In [20]:
# Features importance
importance_3 = model_3.get_booster().get_score(importance_type='gain')

importance_df3 = pd.DataFrame({
    'Feature': list(importance_3.keys()),
    'Importance': list(importance_3.values())
}).sort_values(by='Importance', ascending=False)

importance_df3

,Feature,Importance


In [41]:
xgb_df_3.to_csv('xbg_EB.csv')

## XGB All Features

In [22]:
macro_factor = data.columns[11:]

In [23]:
window_size = 36  # 3 year rolling window
all_predictions_4 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor]
    X_test = test_data[macro_factor].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_4 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
        model_4.fit(X_train, Y_train)
        predicted_returns[etf] = model_4.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    all_predictions_4.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})

# to DataFrame
xgb_df_4 = pd.DataFrame(all_predictions_4)

In [24]:
# Top-4 accuracy
top_4_accuracy4 = []
for i in range(len(xgb_df_4)):
    predicted_set = set(xgb_df_4.iloc[i, 1:5])  # # predicted top 4
    real_set = set(real_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # Top-4 accuracy
    top_4_accuracy4.append(accuracy)

# average Top-4 accuracy
average_top_4_accuracy4 = sum(top_4_accuracy4) / len(top_4_accuracy4)
print("Average Top-4 Accuracy4:", average_top_4_accuracy4)

# print
xgb_df_4['Top-4 Accuracy'] = top_4_accuracy4
print("Predictions with Top-4 Accuracy:")
print(xgb_df_4)

Average Top-4 Accuracy4: 0.38917525773195877
Predictions with Top-4 Accuracy:
          Date Top1 Top2  Top3  Top4  Top-4 Accuracy
0   2008-07-31  XLU  XLI   XLP   XLB            0.50
1   2008-08-31  XLU  XLV   XLE   XLP            0.50
2   2008-09-30  XLV  XLU   XLF   XLK            0.50
3   2008-10-31  XLF  XLB   XLV   XLK            0.50
4   2008-11-30  XLK  XLY   XLI   XLP            0.25
..         ...  ...  ...   ...   ...             ...
189 2024-04-30  XLE  XLC   XLF  XLRE            0.25
190 2024-05-31  XLE  XLY  XLRE   XLC            0.50
191 2024-06-30  XLY  XLE   XLI   XLB            0.25
192 2024-07-31  XLY  XLK   XLB   XLF            0.25
193 2024-08-31  XLY  XLE   XLI   XLC            0.00

[194 rows x 6 columns]


In [25]:
# Features importance
importance_4 = model_4.get_booster().get_score(importance_type='gain')

importance_df4 = pd.DataFrame({
    'Feature': list(importance_4.keys()),
    'Importance': list(importance_4.values())
}).sort_values(by='Importance', ascending=False)

importance_df4.head(20)

,Feature,Importance
33,CPI_for_Urban_Consumers_Gasoline,0.013638
64,Auto_Sales,0.010606
59,FTSE,0.009059
43,Monthly_import_price_index_Capital_goods,0.007146
55,PPI_New_car_dealers,0.003673
63,Housing_Starts,0.003654
32,Export_Air_Freight,0.003507
18,Manufacturers_Motor_Vehicles_And_Parts_Invento...,0.002835
11,Retail_Inventories,0.002513
47,Monthly_Export_price_index_Foods_feeds_and_bev...,0.002490


## LGB Base Feature

In [26]:
import lightgbm as lgb

In [27]:
macro_factor_1 = ['GLD', 'FTSE', 'N225', 'Corprote Bond (BAA) Yield', 'US_1yr',
       'US_10yr', 'Housing Starts', 'Monetary Base', 'M2', 'Unemp_Rate',
       'Auto Sales', 'Durable Goods', 'NYMEX_Oil_Price', 'Michigan_CSI']
macro_factor_1= clean_feature_names(macro_factor_1)

In [28]:
window_size = 48  # 3 year rolling window
lgb_predictions_1 = []
all_real_top4 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor_1]
    X_test = test_data[macro_factor_1].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_lgb_1 = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, verbose= -1)
        model_lgb_1.fit(X_train, Y_train)
        predicted_returns[etf] = model_lgb_1.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    lgb_predictions_1.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})

    # actual rank top 4
    real_top4 = sorted(actual_returns, key=actual_returns.get, reverse=True)[:4]
    all_real_top4.append({'Date': test_data.name, 'Top1': real_top4[0], 'Top2': real_top4[1], 'Top3': real_top4[2], 'Top4': real_top4[3]})


# to DataFrame
lgb_df_1 = pd.DataFrame(lgb_predictions_1)
lgb_top4_df = pd.DataFrame(all_real_top4)

In [29]:
# Top-4 accuracy
lgb_accuracy_1 = []
for i in range(len(lgb_df_1)):
    predicted_set = set(lgb_df_1.iloc[i, 1:5])  # # predicted top 4
    real_set = set(lgb_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # Top-4 accuracy
    lgb_accuracy_1.append(accuracy)

# average Top-4 accuracy
average_top_4_accuracy = sum(lgb_accuracy_1) / len(lgb_accuracy_1)
print("Average Top-4 Accuracy:", average_top_4_accuracy)

# print
lgb_df_1['Top-4 Accuracy'] = lgb_accuracy_1
print("Predictions with Top-4 Accuracy:")
print(lgb_df_1)

Average Top-4 Accuracy: 0.4519230769230769
Predictions with Top-4 Accuracy:
          Date Top1 Top2 Top3  Top4  Top-4 Accuracy
0   2009-07-31  XLB  XLF  XLE   XLK            0.50
1   2009-08-31  XLF  XLB  XLK   XLY            0.50
2   2009-09-30  XLE  XLV  XLB   XLU            0.25
3   2009-10-31  XLU  XLV  XLB   XLY            0.25
4   2009-11-30  XLB  XLP  XLE   XLY            0.50
..         ...  ...  ...  ...   ...             ...
177 2024-04-30  XLK  XLB  XLC   XLP            0.25
178 2024-05-31  XLE  XLC  XLF   XLY            0.25
179 2024-06-30  XLE  XLK  XLY   XLC            0.75
180 2024-07-31  XLK  XLY  XLU   XLI            0.50
181 2024-08-31  XLK  XLY  XLV  XLRE            0.50

[182 rows x 6 columns]


In [42]:
lgb_df_1.set_index('Date').to_csv('lgb_base.csv')

## LGB Engineered Feature

In [31]:
window_size = 48  # 3 year rolling window
lgb_predictions_2 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor_2]
    X_test = test_data[macro_factor_2].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_lgb_2 = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, verbose= -1)
        model_lgb_2.fit(X_train, Y_train)
        predicted_returns[etf] = model_lgb_2.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    lgb_predictions_2.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})


# to DataFrame
lgb_df_2 = pd.DataFrame(lgb_predictions_2)

In [32]:
# Top-4 accuracy
lgb_accuracy_2 = []
for i in range(len(lgb_df_2)):
    predicted_set = set(lgb_df_2.iloc[i, 1:5])  # # predicted top 4
    real_set = set(lgb_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # Top-4 accuracy
    lgb_accuracy_2.append(accuracy)

# average Top-4 accuracy
average_top_4_accuracy = sum(lgb_accuracy_2) / len(lgb_accuracy_2)
print("Average Top-4 Accuracy:", average_top_4_accuracy)

# print
lgb_df_2['Top-4 Accuracy'] = lgb_accuracy_2
print("Predictions with Top-4 Accuracy:")
print(lgb_df_2)

Average Top-4 Accuracy: 0.3942307692307692
Predictions with Top-4 Accuracy:
          Date Top1  Top2 Top3 Top4  Top-4 Accuracy
0   2009-07-31  XLE   XLK  XLB  XLF            0.50
1   2009-08-31  XLE   XLK  XLP  XLU            0.00
2   2009-09-30  XLE   XLV  XLP  XLB            0.25
3   2009-10-31  XLB   XLP  XLV  XLK            0.75
4   2009-11-30  XLB   XLK  XLP  XLF            0.25
..         ...  ...   ...  ...  ...             ...
177 2024-04-30  XLF  XLRE  XLE  XLI            0.50
178 2024-05-31  XLE  XLRE  XLC  XLF            0.50
179 2024-06-30  XLF   XLI  XLC  XLK            0.50
180 2024-07-31  XLC   XLK  XLF  XLI            0.50
181 2024-08-31  XLK   XLC  XLF  XLI            0.00

[182 rows x 6 columns]


## LGB Engineered&Base Feature

In [33]:
window_size = 48  # 3 year rolling window
lgb_predictions_3 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor_3]
    X_test = test_data[macro_factor_3].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_lgb_3 = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, verbose= -1)
        model_lgb_3.fit(X_train, Y_train)
        predicted_returns[etf] = model_lgb_3.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    lgb_predictions_3.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})


# to DataFrame
lgb_df_3 = pd.DataFrame(lgb_predictions_3)

In [34]:
# Top-4 accuracy
lgb_accuracy_3 = []
for i in range(len(lgb_df_3)):
    predicted_set = set(lgb_df_3.iloc[i, 1:5])  # # predicted top 4
    real_set = set(lgb_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # Top-4 accuracy
    lgb_accuracy_3.append(accuracy)

# average Top-4 accuracy
average_top_4_accuracy = sum(lgb_accuracy_3) / len(lgb_accuracy_3)
print("Average Top-4 Accuracy:", average_top_4_accuracy)

# print
lgb_df_3['Top-4 Accuracy'] = lgb_accuracy_3
print("Predictions with Top-4 Accuracy:")
print(lgb_df_3)

Average Top-4 Accuracy: 0.4423076923076923
Predictions with Top-4 Accuracy:
          Date Top1 Top2  Top3  Top4  Top-4 Accuracy
0   2009-07-31  XLE  XLB   XLK   XLI            0.50
1   2009-08-31  XLK  XLB   XLI   XLE            0.25
2   2009-09-30  XLE  XLV   XLB   XLK            0.50
3   2009-10-31  XLV  XLE   XLU   XLP            0.75
4   2009-11-30  XLP  XLE   XLU   XLY            0.25
..         ...  ...  ...   ...   ...             ...
177 2024-04-30  XLB  XLF   XLE  XLRE            0.25
178 2024-05-31  XLC  XLE   XLK   XLY            0.50
179 2024-06-30  XLK  XLC   XLY   XLI            0.75
180 2024-07-31  XLK  XLC   XLY   XLU            0.25
181 2024-08-31  XLK  XLC  XLRE   XLY            0.25

[182 rows x 6 columns]


In [35]:
lgb_df_3.set_index('Date').to_csv('lgb_EB.csv')

## LGB All features

In [36]:
window_size = 48  # 3 year rolling window
lgb_predictions_4 = []

# rolling window prediction
for i in range(window_size, len(data) - 1):
    train_data = data.iloc[i - window_size:i]  # training
    test_data = data.iloc[i + 1]  # testing
    
    X_train = train_data[macro_factor]
    X_test = test_data[macro_factor].values.reshape(1, -1)  # testing features

    # predict etf return
    predicted_returns = {}
    actual_returns = test_data[etf_columns].to_dict()  # actual return
    
    for etf in etf_columns:
        Y_train = train_data[etf]  # set return as the target
        model_lgb_4 = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, verbose= -1)
        model_lgb_4.fit(X_train, Y_train)
        predicted_returns[etf] = model_lgb_4.predict(X_test)[0]  # predict return

    # predict rank top 4
    predicted_top4 = sorted(predicted_returns, key=predicted_returns.get, reverse=True)[:4]
    lgb_predictions_4.append({'Date': test_data.name, 'Top1': predicted_top4[0], 'Top2': predicted_top4[1], 'Top3': predicted_top4[2], 'Top4': predicted_top4[3]})


# to DataFrame
lgb_df_4 = pd.DataFrame(lgb_predictions_4)

In [37]:
# Top-4 accuracy
lgb_accuracy_4 = []
for i in range(len(lgb_df_4)):
    predicted_set = set(lgb_df_4.iloc[i, 1:5])  # # predicted top 4
    real_set = set(lgb_top4_df.iloc[i, 1:5])  # actual top 4
    match_count = len(predicted_set & real_set)
    accuracy = match_count / 4  # Top-4 accuracy
    lgb_accuracy_4.append(accuracy)

# average Top-4 accuracy
average_top_4_accuracy = sum(lgb_accuracy_4) / len(lgb_accuracy_4)
print("Average Top-4 Accuracy:", average_top_4_accuracy)

# print
lgb_df_4['Top-4 Accuracy'] = lgb_accuracy_4
print("Predictions with Top-4 Accuracy:")
print(lgb_df_4)

Average Top-4 Accuracy: 0.40934065934065933
Predictions with Top-4 Accuracy:
          Date  Top1 Top2 Top3  Top4  Top-4 Accuracy
0   2009-07-31   XLF  XLY  XLB   XLI            1.00
1   2009-08-31   XLY  XLK  XLB   XLV            0.50
2   2009-09-30   XLY  XLK  XLB   XLF            0.50
3   2009-10-31   XLV  XLF  XLU   XLP            0.50
4   2009-11-30   XLE  XLP  XLK   XLU            0.00
..         ...   ...  ...  ...   ...             ...
177 2024-04-30  XLRE  XLK  XLE   XLF            0.25
178 2024-05-31   XLK  XLI  XLC  XLRE            0.75
179 2024-06-30   XLK  XLY  XLB   XLC            0.75
180 2024-07-31   XLK  XLC  XLY   XLI            0.25
181 2024-08-31   XLK  XLC  XLY   XLB            0.00

[182 rows x 6 columns]


In [38]:
data[macro_factor_3]

,GLD,FTSE,N225,Corprote_Bond_(BAA)_Yield,US_1yr,US_10yr,CPI,Housing_Starts,Monetary_Base,M2,...,CPI_for_Urban_Consumers_Energy,CPI_for_Urban_Consumers_Housing,PPI_Manufacturing_Industries,PPI_All_Commodities,Monthly_Export_price_index_Foods_feeds_and_beverages,Monthly_Export_price_index_Consumer_goods_excluding_automotives,PPI_Iron_and_steel_mills_and_ferroalloy,PPI_Aircraft_engine_and_engine_parts,PPI_sub_sector_data_for_Truck_transportation,PPI_General_medical_and_surgical_hospitals
Date,,,,,,,,,,,,,,,,,,,,,
2005-06-30,43.35,4801.7,11008.90,6.05,3.35,4.50,193.10,1864.0,773.5,6441.9,...,170.9,194.4,149.6,155.0,121.0,101.9,151.3,163.5,107.6,146.1
2005-07-31,41.65,4964.0,11276.59,6.01,3.33,4.21,193.70,2061.0,775.5,6455.9,...,169.4,194.5,149.4,154.3,123.6,101.7,147.1,163.6,108.1,146.2
2005-08-31,43.44,5113.2,11584.01,5.86,3.32,4.00,193.60,2025.0,776.4,6473.3,...,171.4,195.5,149.6,154.3,125.2,101.5,141.5,163.5,108.7,146.3
2005-09-30,42.82,5282.3,11899.60,5.95,3.45,3.94,193.70,2068.0,780.7,6505.8,...,178.5,196.6,151.0,156.3,125.4,101.5,139.3,163.3,109.1,146.6
2005-10-31,43.40,5296.9,12413.60,5.96,3.80,4.28,194.90,2054.0,782.9,6537.4,...,186.6,196.9,151.8,157.6,124.9,101.6,133.7,163.4,109.4,146.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,189.31,7630.0,39166.19,5.77,4.73,3.99,309.69,1376.0,5843.7,20751.4,...,276.3,329.7,248.3,254.9,230.7,117.4,292.4,269.0,184.7,235.8
2024-05-31,205.72,7952.6,40369.44,5.75,5.01,4.25,311.05,1546.0,5896.9,20744.4,...,285.0,331.2,249.5,255.1,227.9,117.9,267.0,269.0,182.8,235.7
2024-06-30,211.87,8144.1,38405.66,6.00,5.03,4.20,312.23,1299.0,5883.0,20836.0,...,290.8,331.7,251.4,257.0,227.4,118.0,259.7,269.2,183.7,235.8


In [39]:
data[etf_columns]

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2005-06-30,-0.019912,-100.000000,0.069554,0.012508,-0.026557,-0.013453,-0.019244,-100.000000,0.059829,-0.005336,-0.004277
2005-07-31,0.055305,-100.000000,0.071262,0.015442,0.038194,0.057143,0.031977,-100.000000,0.022333,0.022798,0.056619
2005-08-31,-0.041711,-100.000000,0.063613,-0.016423,-0.023411,-0.009828,-0.017606,-100.000000,0.006675,-0.003497,-0.032890
2005-09-30,0.007254,-100.000000,0.062884,0.008658,0.016634,0.002481,0.014337,-100.000000,0.042194,-0.005702,-0.028277
2005-10-31,0.002216,-100.000000,-0.090354,0.030656,-0.014437,-0.022896,-0.004947,-100.000000,-0.064199,-0.030437,-0.020448
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,-0.045850,-0.046552,-0.009366,-0.041627,-0.035246,-0.057582,-0.011275,-0.084406,0.016540,-0.050133,-0.044998
2024-05-31,0.032680,0.069362,-0.003478,0.031702,0.016403,0.070764,0.024282,0.051433,0.089568,0.024056,0.002001
2024-06-30,-0.030544,0.031526,-0.014177,-0.008952,-0.009862,0.078312,-0.002227,0.019513,-0.055408,0.018232,0.038749
